In [ ]:
from __future__ import print_function
import random, numpy as np
import keras, tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils import image_dataset_from_directory
from keras.layers import Dense, Dropout, Activation, Flatten, Rescaling, RandomRotation, RandomFlip, RandomCrop, RandomTranslation
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import *

from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
%matplotlib inline

## Set random seed

In [ ]:
import random, os, tensorflow as tf

def set_seed(seed_value=12345):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # to Disable GPU training

In [ ]:
print('Default GPU Device:', tf.test.gpu_device_name() or 'None')

In [ ]:
data = './KanjiN5_normalize'

## Model Training

In [ ]:
num_classes = 81
batch_size = 64
image_size = 120

In [ ]:
data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
                                                height_shift_range=0.1,
                                                width_shift_range=0.1,
                                                zoom_range=0.1,
                                                rescale=1./255
)

In [ ]:
train_ds = data_augmentation.flow_from_directory(
    directory=os.path.join(data, "Train"),
    class_mode='categorical',
    batch_size=batch_size,
    target_size=(image_size, image_size),
    shuffle=True
)

valid_ds = data_augmentation.flow_from_directory(
    directory=os.path.join(data, "Valid"),
    class_mode='categorical',
    batch_size=batch_size,
    target_size=(image_size, image_size),
    shuffle=False
)

test_ds = data_augmentation.flow_from_directory(
    directory=os.path.join(data, "Test"),
    class_mode='categorical',
    batch_size=batch_size,
    target_size=(image_size, image_size),
    shuffle=False
)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Rescaling
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(6, (5,5), input_shape=(image_size, image_size, 3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(16, (5,5), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
callback = [keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')]
# https://stackoverflow.com/questions/73304934/tensorflow-data-augmentation-gives-a-warning-using-a-while-loop-for-converting
# https://medium.com/featurepreneur/data-augmentation-using-keras-preprocessing-layers-6cdc7d49328e

In [ ]:
epochs = 100
earlystop = 1
if earlystop:
    history = model.fit(train_ds, epochs=epochs, validation_data=valid_ds, callbacks=callback)
else:
    history = model.fit(train_ds, epochs=epochs, validation_data=valid_ds)

## Loss visualization 

In [ ]:
# Loss accuracy
fig = plt.figure(figsize=(14, 5), dpi=80)
ax = fig.add_subplot(1, 2, 1)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.plot(history.history['loss'])
ax.plot(history.history['val_loss'])
ax.set_title('Loss')
ax.set_ylabel('loss')
ax.set_xlabel('epoch')
ax.legend(['train', 'val'], loc='upper right')

ax = fig.add_subplot(1, 2, 2)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.plot(history.history['accuracy'])

ax.plot(history.history['val_accuracy'])
ax.set_title('Accuracy')
ax.set_ylabel('accuracy')
ax.set_xlabel('epoch')
ax.legend(['train', 'val'], loc='lower right')

In [ ]:
print('Train Loss: {:.6f}, Accuracy: {:.6f}'.format(*model.evaluate(train_ds, verbose=0)))
print('Validation Loss: {:.6f}, Accuracy: {:.6f}'.format(*model.evaluate(valid_ds, verbose=0)))
print('Test Loss: {:.6f}, Accuracy: {:.6f}'.format(*model.evaluate(test_ds, verbose=0)))



In [ ]:

from sklearn.metrics import classification_report
 

## Create Confusion Matrix
using seaborn and matplotlib.

In [ ]:
# Generate predictions
predictions = model.predict(test_ds)
y_pred = np.argmax(predictions, axis=1)

# Get true labels
true_labels = test_ds.classes



In [ ]:
# Generate classification report
report = classification_report(true_labels, y_pred, target_names=list(train_ds.class_indices.keys()))
print("Classification Report:")
print(report)

In [ ]:
y_pred = model.predict(test_ds)
y_pred = y_pred.argmax(axis=1)
print(y_pred[0:10], type(y_pred))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Generate confusion matrix
cm = confusion_matrix(true_labels, y_pred)
# Define class labels (assuming your classes are named '0', '1', ..., '19')
class_names = [i for i in range(80)]

# Plot confusion matrix
plt.figure(figsize=(18, 13))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()


In [ ]:
model.save('Lenet5_Dropout2.h5')